In [ ]:
# 라이브러리 import 
import pandas as pd
import numpy as np

import lightgbm as lgbm
from sklearn.metrics import roc_auc_score, roc_curve, log_loss
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold

import sklearn.metrics


In [ ]:
# 파이썬 버전 및 라이브러리 버전

print('pandas',pd.__version__)
print('numpy',np.__version__)
print('sklearn.metrics',sklearn.__version__)
print('python',sys.version)

pandas 1.0.5
numpy 1.18.5
sklearn.metrics 0.23.1
python 3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]



# 데이터 불러오기


In [ ]:
# 데이터 불러오기

train = pd.read_csv('CE_train.csv', header=0)
test = pd.read_csv('CE_test.csv', header=0)

train = train.drop(['Unnamed: 0'] , axis = 'columns')
test = test.drop(['Unnamed: 0'] , axis = 'columns')


# 데이터 전처리 및 모델링


In [ ]:
# bad_features 제거 : permutation importance 기준

train = train.drop(["dered_g","TS4","dered_gi_differ","p1","dered_c_paral","dered_c3","TS11","dered_TS8","dered_criteria1","dered_criteria4"
    ,"c1","TS6","airmass_g","dered_ri_differ","gi_differ","TS2","icol","c_paral_gaus","ELG_dered_differ1","dered_TS7","dered_gr_differ"
    ,"criteria3","scol","ELG_differ1","TS1","TS7","airmass_i","airmass_r","dered_TS1","c_paral","airmass_u","ELG_differ2","ri_differ"
    ,"TS9","TS14","airmass_z","TS8","dered_criteria2","gr_differ","dered_TS10","criterion2","TS10","dered_criteria3","c3",'TS13','criteria4'
    ,'dered_c_paral_gaus'] , axis = 'columns')
                   

test = test.drop(["dered_g","TS4","dered_gi_differ","p1","dered_c_paral","dered_c3","TS11","dered_TS8","dered_criteria1","dered_criteria4"
    ,"c1","TS6","airmass_g","dered_ri_differ","gi_differ","TS2","icol","c_paral_gaus","ELG_dered_differ1","dered_TS7","dered_gr_differ"
    ,"criteria3","scol","ELG_differ1","TS1","TS7","airmass_i","airmass_r","dered_TS1","c_paral","airmass_u","ELG_differ2","ri_differ"
    ,"TS9","TS14","airmass_z","TS8","dered_criteria2","gr_differ","dered_TS10","criterion2","TS10","dered_criteria3","c3",'TS13','criteria4'
    ,'dered_c_paral_gaus'] , axis = 'columns')
                   

In [ ]:
# 휴리스틱하게 찾은 변수 추가 생성 : 의외로 성능 상승

def red(train):
    if train['redshift'] < 0:
        return 0
    else:
        return 1
    
    
train['grade'] = train.apply(red, axis=1)
test['grade'] = test.apply(red, axis=1)

train['red2'] = train['redshift'] % 2
test['red2'] = test['redshift'] % 2

train['red3'] = train['redshift'] % 10
test['red3'] = test['redshift'] % 10

train['red4'] = train['redshift'] % 100
test['red4'] = test['redshift'] % 100

In [ ]:
# 모델링을 위한 전작업
print(len(train.columns))
train_columns = [c for c in train.columns if c not in ['class']]
num_class = train['class'].nunique()
print(len(train_columns))

91
90


In [ ]:
x_train = train.copy()
y_train = train['class']
x_test = test.copy()

In [ ]:
# 5fold LGBM 모델링
# gbdt가 가장 성능이 좋았음
# 검색을 통해 얻어진 파라미터가 1차적 성능을 보기위해 사용됐는데 
# hyperobt로 찾은 파라미터 보다 성능이 좋아 사용 

print('LightGBM',lgbm.__version__)

lgb_param = {'objective': 'multiclass', 
 'num_class': 3, 
 'boosting_type': 'gbdt', 
 'subsample_freq': 5, 
 'num_leaves': 92, 
 'min_data_in_leaf': 64, 
 'subsample_for_bin': 23000, 
 'max_depth': 10, 
 'feature_fraction': 0.302, 
 'bagging_fraction': 0.904, 
 'lambda_l1': 0.099, 
 'lambda_l2': 1.497, 
 'min_child_weight': 38.011, 
 'nthread': 32, 
 'metric': 'multi_logloss', 
 'learning_rate': 0.021, 
 'min_sum_hessian_in_leaf': 3, 
 'seed': 42, 
 'n_estimators': 10000}




num_class = 3
oof_train = np.zeros((len(x_train), num_class))
oof_test = np.zeros((len(x_test), num_class))
log_loss_score_list = []
NFOLD = 5
SEED = 42




# stratifiedkfold 5 fold 사용
folds = StratifiedKFold(n_splits=NFOLD, shuffle=True, random_state=42)
for fold_, (trn_index, val_index) in enumerate(folds.split(x_train, y_train)):
    print(f"{fold_ + 1} FOLD Start!!")
    trn_x, trn_y = x_train.iloc[trn_index][train_columns], y_train.iloc[trn_index]
    val_x, val_y = x_train.iloc[val_index][train_columns], y_train.iloc[val_index]
    dtrain = lgbm.Dataset(trn_x, label=trn_y, silent=True)
    dcross = lgbm.Dataset(val_x, label=val_y, silent=True)

    # dart는 얼리스탑핑이 안되서 한번 num_boost_round를 넉넉히 돌린다음에 5fold에서 가장 좋았던 round로 고정하고 돌린다.
    clf = lgbm.train(lgb_param, train_set=dtrain, num_boost_round=10000, valid_sets=[dtrain, dcross],
                     early_stopping_rounds=100, verbose_eval=100, categorical_feature='')

    val_pred = clf.predict(val_x)
    oof_train[val_index, :] = val_pred

    log_loss_score = log_loss(val_y, val_pred)
    log_loss_score_list.append(log_loss_score)
    print(f"{fold_ + 1} FOLD LogLoss: ", log_loss_score)

    # 5fold 평균으로 제출
    oof_test += clf.predict(x_test[train_columns]) / NFOLD



print(log_loss_score_list)
np.mean(log_loss_score_list), np.std(log_loss_score_list)


sample_submission = pd.read_csv('sample_submission2.csv')


submission = pd.DataFrame(data = oof_test, columns = sample_submission.columns[1:], index = sample_submission['id'])
submission.to_csv('submission.csv', index=True)



1 FOLD Start!!


Found `n_estimators` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is []


Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.244188	valid_1's multi_logloss: 0.248515
[200]	training's multi_logloss: 0.163564	valid_1's multi_logloss: 0.172407
[300]	training's multi_logloss: 0.144954	valid_1's multi_logloss: 0.158162
[400]	training's multi_logloss: 0.136996	valid_1's multi_logloss: 0.154194
[500]	training's multi_logloss: 0.131619	valid_1's multi_logloss: 0.152828
[600]	training's multi_logloss: 0.127427	valid_1's multi_logloss: 0.152247
[700]	training's multi_logloss: 0.123565	valid_1's multi_logloss: 0.151943
[800]	training's multi_logloss: 0.119938	valid_1's multi_logloss: 0.151815
[900]	training's multi_logloss: 0.116532	valid_1's multi_logloss: 0.15172
[1000]	training's multi_logloss: 0.113293	valid_1's multi_logloss: 0.151616
[1100]	training's multi_logloss: 0.110312	valid_1's multi_logloss: 0.151579
Early stopping, best iteration is:
[1098]	training's multi_logloss: 0.110376	valid_1's multi_logloss: 0.151574
1